<h1 style="text-align:center"> INFO 323: Cloud Computing and Big Data</h1>
<h2 style="text-align:center"> College of Computing and Informatics</h2>
<h2 style="text-align:center">Drexel University</h2>

<h3 style="text-align:center"> Aggregations</h3>
<h3 style="text-align:center"> Yuan An, PhD</h3>
<h3 style="text-align:center">Associate Professor</h3>

In [0]:
spark.version

Out[1]: '3.1.2'

Let’s begin by reading in our data on purchases, repartitioning the data to have far fewer partitions
(because we know it’s a small volume of data stored in a lot of small files), and caching the results
for rapid access:

In [0]:
filepath = "dbfs:/FileStore/tables/retail-data/by-day/*.csv"

In [0]:
df = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load(filepath)

In [0]:
# get the number of partitions of the DataFrame
df.rdd.getNumPartitions()

Out[4]: 10

In [0]:
# Coalesce to 5 partitions
df = df.coalesce(5)
df.rdd.getNumPartitions()

Out[5]: 5

In [0]:
df.cache()
df.createOrReplaceTempView("dfTable")

## count
The first function worth going over is count, except in this example it will perform as a
transformation instead of an action. In this case, we can do one of two things: specify a specific
column to count, or all the columns by using count(*) or count(1) to represent that we want to
count every row as the literal one, as shown in this example:

In [0]:
# COMMAND ----------

from pyspark.sql.functions import count
df.select(count("StockCode")).show() # 541909

+----------------+
count(StockCode)|
+----------------+
 541909|
+----------------+

## countDistinct
Sometimes, the total number is not relevant; rather, it’s the number of unique groups that you want. To
get this number, you can use the countDistinct function. This is a bit more relevant for individual
columns:

In [0]:
# COMMAND ----------

from pyspark.sql.functions import countDistinct
df.select(countDistinct("StockCode")).show() # 4070

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



## approx_count_distinct
Often, we find ourselves working with large datasets and the exact distinct count is irrelevant. There
are times when an approximation to a certain degree of accuracy will work just fine, and for that, you
can use the approx_count_distinct function:

In [0]:
# COMMAND ----------

from pyspark.sql.functions import approx_count_distinct
df.select(approx_count_distinct("StockCode", 0.1)).show() # 3364

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+



## first and last
You can get the first and last values from a DataFrame by using these two obviously named functions.
This will be based on the rows in the DataFrame, not on the values in the DataFrame:

In [0]:
# COMMAND ----------

from pyspark.sql.functions import first, last
df.select(first("StockCode"), last("StockCode")).show()

+----------------+---------------+
|first(StockCode)|last(StockCode)|
+----------------+---------------+
|           23084|          22168|
+----------------+---------------+



In [0]:
# get the first row


Out[16]: Row(InvoiceNo='580538', StockCode='23084', Description='RABBIT NIGHT LIGHT', Quantity=48, InvoiceDate=datetime.datetime(2011, 12, 5, 8, 38), UnitPrice=1.79, CustomerID=14075.0, Country='United Kingdom')

In [0]:
# get the Quantity of the first row


## min and max
To extract the minimum and maximum values from a DataFrame, use the min and max functions:

In [0]:
# COMMAND ----------

from pyspark.sql.functions import min, max
df.select(min("Quantity"), max("Quantity")).show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



## sum
Another simple task is to add all the values in a row using the sum function:

In [0]:
# COMMAND ----------

from pyspark.sql.functions import sum
df.select(sum("Quantity")).show() # 5176450

+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+



## sumDistinct
In addition to summing a total, you also can sum a distinct set of values by using the sumDistinct
function:

In [0]:
# COMMAND ----------

from pyspark.sql.functions import sumDistinct
df.select(sumDistinct("Quantity")).show() # 29310

+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|                 29310|
+----------------------+



## avg
Although you can calculate average by dividing sum by count, Spark provides an easier way to get
that value via the avg or mean functions. In this example, we use alias in order to more easily reuse
these columns later:

In [0]:
# COMMAND ----------

from pyspark.sql.functions import sum, count, avg, expr

df.select(
    count("Quantity").alias("total_transactions"),
    sum("Quantity").alias("total_purchases"),
    avg("Quantity").alias("avg_purchases"),
    expr("mean(Quantity)").alias("mean_purchases"))\
  .selectExpr(
    "total_purchases/total_transactions",
    "avg_purchases",
    "mean_purchases").show()

+--------------------------------------+----------------+----------------+
|(total_purchases / total_transactions)|   avg_purchases|  mean_purchases|
+--------------------------------------+----------------+----------------+
|                      9.55224954743324|9.55224954743324|9.55224954743324|
+--------------------------------------+----------------+----------------+



## Variance and Standard Deviation
Calculating the mean naturally brings up questions about the variance and standard deviation.  You
can calculate these in Spark by using their respective functions. However, something to note is that
Spark has both the formula for the sample standard deviation as well as the formula for the population
standard deviation. These are fundamentally different statistical formulae, and we need to
differentiate between them. By default, Spark performs the formula for the sample standard deviation
or variance if you use the variance or stddev functions.

In [0]:
# COMMAND ----------

from pyspark.sql.functions import var_pop, stddev_pop
from pyspark.sql.functions import var_samp, stddev_samp
df.select(var_pop("Quantity"), var_samp("Quantity"),
  stddev_pop("Quantity"), stddev_samp("Quantity")).show()

+-----------------+------------------+--------------------+---------------------+
|var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+-----------------+------------------+--------------------+---------------------+
|47559.30364660885| 47559.39140929855|   218.0809566344775|    218.0811578502337|
+-----------------+------------------+--------------------+---------------------+



## skewness and kurtosis
Skewness and kurtosis are both measurements of extreme points in your data. Skewness measures the
asymmetry of the values in your data around the mean, whereas kurtosis is a measure of the tail of
data. These are both relevant specifically when modeling your data as a probability distribution of a
random variable. Although here we won’t go into the math behind these specifically, you can look up
definitions quite easily on the internet. You can calculate these by using the functions:

In [0]:
# COMMAND ----------

from pyspark.sql.functions import skewness, kurtosis
df.select(skewness("Quantity"), kurtosis("Quantity")).show()

+------------------+------------------+
|skewness(Quantity)|kurtosis(Quantity)|
+------------------+------------------+
|-0.264075576105286|119768.05495534562|
+------------------+------------------+



## Covariance and Correlation
We discussed single column aggregations, but some functions compare the interactions of the values
in two difference columns together. Two of these functions are cov and corr, for covariance and
correlation, respectively. Correlation measures the Pearson correlation coefficient, which is scaled
between –1 and +1. The covariance is scaled according to the inputs in the data.

Like the var function, covariance can be calculated either as the sample covariance or the population
covariance. Therefore it can be important to specify which formula you want to use. Correlation has
no notion of this and therefore does not have calculations for population or sample. Here’s how they
work:

In [0]:
# COMMAND ----------

from pyspark.sql.functions import corr, covar_pop, covar_samp
df.select(corr("InvoiceNo", "Quantity"), covar_samp("InvoiceNo", "Quantity"),
    covar_pop("InvoiceNo", "Quantity")).show()

+-------------------------+-------------------------------+------------------------------+
|corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
|     4.912186085617365E-4|             1052.7280543863135|            1052.7260778702093|
+-------------------------+-------------------------------+------------------------------+



## Aggregating to Complex Types
In Spark, you can perform aggregations not just of numerical values using formulas, you can also
perform them on complex types. For example, we can collect a list of values present in a given
column or only the unique values by collecting to a set.
You can use this to carry out some more programmatic access later on in the pipeline or pass the
entire collection in a user-defined function (UDF):

In [0]:
# COMMAND ----------

from pyspark.sql.functions import collect_set, collect_list
df.agg(collect_set("Country"), collect_list("Country")).show(1)

+--------------------+---------------------+
|collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
|[Portugal, Italy,...| [United Kingdom, ...|
+--------------------+---------------------+



## Grouping
Thus far, we have performed only DataFrame-level aggregations. A more common task is to perform
calculations based on groups in the data. This is typically done on categorical data for which we
group our data on one column and perform some calculations on the other columns that end up in that
group.

The best way to explain this is to begin performing some groupings. The first will be a count, just as
we did before. We will group by each unique invoice number and get the count of items on that
invoice. Note that this returns another DataFrame and is lazily performed.

We do this grouping in two phases. First we specify the column(s) on which we would like to group,
and then we specify the aggregation(s). The first step returns a RelationalGroupedDataset, and the
second step returns a DataFrame.

As mentioned, we can specify any number of columns on which we want to group:

In [0]:
# COMMAND ----------

from pyspark.sql.functions import count
df.groupBy("InvoiceNo", "CustomerId").count().show()

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   580657|   14696.0|   20|
|   581474|   12748.0|   24|
|   576641|   17549.0|    4|
|   575753|   17841.0|   80|
|   576357|   15660.0|   18|
|  C579018|   16678.0|    1|
|   579062|   17651.0|   16|
|   570413|   17652.0|   18|
|  C569961|   13704.0|    1|
|   572518|   16353.0|    1|
|   577696|   16406.0|   63|
|   577783|   18139.0|   35|
|   580090|   13323.0|    5|
|   577338|   17650.0|    8|
|  C577375|   14441.0|    2|
|   568670|   14978.0|   38|
|   580138|   13199.0|    3|
|  C538084|   12649.0|    2|
|  C538085|   16350.0|    3|
|   575200|   12921.0|    1|
+---------+----------+-----+
only showing top 20 rows



## Grouping with Expressions
As we saw earlier, counting is a bit of a special case because it exists as a method. For this, usually
we prefer to use the count function. Rather than passing that function as an expression into a select
statement, we specify it as within agg. This makes it possible for you to pass-in arbitrary expressions
that just need to have some aggregation specified. You can even do things like alias a column after
transforming it for later use in your data flow:

In [0]:
df.groupBy("InvoiceNo").agg(
    count("Quantity").alias("quan"),
    expr("count(Quantity)")).show()

+---------+----+---------------+
|InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
|   574966|   8|              8|
|   575091|  38|             38|
|   578057|  28|             28|
|   537252|   1|              1|
|   578459|   8|              8|
|  C578132|   1|              1|
|   578292|  72|             72|
|   576112|  20|             20|
|   577022|  38|             38|
|   574592|   8|              8|
|  C576393|   2|              2|
|   577511|  46|             46|
|   577541|  21|             21|
|   580739|   2|              2|
|   580906|   4|              4|
|   573726|   1|              1|
|   575671|  20|             20|
|   570264|   1|              1|
|   570281|   3|              3|
|   569823|  69|             69|
+---------+----+---------------+
only showing top 20 rows



## Grouping Sets
We may want an aggregation across multiple groups. We achieve this by using grouping sets. Grouping
sets are a low-level tool for combining sets of aggregations together. They give you the ability to
create arbitrary aggregation in their group-by statements.

Let’s work through an example to gain a better understanding. Here, we would like to get the total
quantity of all stock codes and customers.

In [0]:
# COMMAND ----------

dfNoNull = dfWithDate.drop()
dfNoNull.createOrReplaceTempView("dfNoNull")

```
-- in SQL
SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNull
GROUP BY customerId, stockCode
ORDER BY CustomerId DESC, stockCode DESC
```

You can do the exact same thing by using a grouping set:

```
-- in SQL
SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNull
GROUP BY customerId, stockCode GROUPING SETS((customerId, stockCode))
ORDER BY CustomerId DESC, stockCode DESC
```

Simple enough, but what if you also want to include the total number of items, regardless of customer
or stock code? With a conventional group-by statement, this would be impossible. But, it’s simple
with grouping sets: we simply specify that we would like to aggregate at that level, as well, in our
grouping set. This is, effectively, the union of several different groupings together:

```
-- in SQL
SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNull
GROUP BY customerId, stockCode GROUPING SETS((customerId, stockCode),())
ORDER BY CustomerId DESC, stockCode DESC
```

The GROUPING SETS operator is only available in SQL. To perform the same in DataFrames, you use
the rollup and cube operators—which allow us to get the same results. Let’s go through those.

## Rollups
A rollup is a multidimensional aggregation that performs a variety of group-by style calculations for us.
Let’s create a rollup that looks across time (with our new Date column) and space (with the Country
column) and creates a new DataFrame that includes the grand total over all dates, the grand total for
each date in the DataFrame, and the subtotal for each country on each date in the DataFrame:

In [0]:
# COMMAND ----------

rolledUpDF = dfNoNull.rollup("Date", "Country").agg(sum("Quantity"))\
  .selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity")\
  .orderBy("Date")
rolledUpDF.show()

+----------+--------------+--------------+
|      Date|       Country|total_quantity|
+----------+--------------+--------------+
|      null|          null|       5176450|
|2010-12-01|   Netherlands|            97|
|2010-12-01|          EIRE|           243|
|2010-12-01|       Germany|           117|
|2010-12-01|        France|           449|
|2010-12-01|     Australia|           107|
|2010-12-01|          null|         26814|
|2010-12-01|United Kingdom|         23949|
|2010-12-01|        Norway|          1852|
|2010-12-02|          EIRE|             4|
|2010-12-02|          null|         21023|
|2010-12-02|       Germany|           146|
|2010-12-02|United Kingdom|         20873|
|2010-12-03|        Poland|           140|
|2010-12-03|   Switzerland|           110|
|2010-12-03|       Germany|           170|
|2010-12-03|         Spain|           400|
|2010-12-03|         Italy|           164|
|2010-12-03|      Portugal|            65|
|2010-12-03|          null|         14830|
+----------

Now where you see the null values is where you’ll find the grand totals. A null in both rollup
columns specifies the grand total across both of those columns:

In [0]:
rolledUpDF.where("Country IS NULL").show()
rolledUpDF.where("Date IS NULL").show()

+----------+-------+--------------+
|      Date|Country|total_quantity|
+----------+-------+--------------+
|      null|   null|       5176450|
|2010-12-01|   null|         26814|
|2010-12-02|   null|         21023|
|2010-12-03|   null|         14830|
|2010-12-05|   null|         16395|
|2010-12-06|   null|         21419|
|2010-12-07|   null|         24995|
|2010-12-08|   null|         22741|
|2010-12-09|   null|         18431|
|2010-12-10|   null|         20297|
|2010-12-12|   null|         10565|
|2010-12-13|   null|         17623|
|2010-12-14|   null|         20098|
|2010-12-15|   null|         18229|
|2010-12-16|   null|         29632|
|2010-12-17|   null|         16069|
|2010-12-19|   null|          3795|
|2010-12-20|   null|         14965|
|2010-12-21|   null|         15467|
|2010-12-22|   null|          3192|
+----------+-------+--------------+
only showing top 20 rows

+----+-------+--------------+
|Date|Country|total_quantity|
+----+-------+--------------+
|null|   null|      

## Cube
A cube takes the rollup to a level deeper. Rather than treating elements hierarchically, a cube does the
same thing across all dimensions. This means that it won’t just go by date over the entire time period,
but also the country.

To pose this as a question again, can you make a table that includes the
following?
* The total across all dates and countries
* The total for each date across all countries
* The total for each country on each date
* The total for each country across all dates

The method call is quite similar, but instead of calling rollup, we call cube:

In [0]:
# COMMAND ----------

from pyspark.sql.functions import sum

dfNoNull.cube("Date", "Country").agg(sum(col("Quantity")))\
  .select("Date", "Country", "sum(Quantity)").orderBy("Date").show()

+----+--------------------+-------------+
|Date|             Country|sum(Quantity)|
+----+--------------------+-------------+
|null|               Japan|        25218|
|null|           Australia|        83653|
|null|              Cyprus|         6317|
|null|         Unspecified|         3300|
|null|            Portugal|        16180|
|null|                 RSA|          352|
|null|             Germany|       117448|
|null|     Channel Islands|         9479|
|null|           Singapore|         5234|
|null|                null|      5176450|
|null|                 USA|         1034|
|null|             Finland|        10666|
|null|           Hong Kong|         4769|
|null|             Lebanon|          386|
|null|               Spain|        26824|
|null|  European Community|          497|
|null|             Denmark|         8188|
|null|              Norway|        19247|
|null|United Arab Emirates|          982|
|null|      Czech Republic|          592|
+----+--------------------+-------